### Install and import needed packages

In [2]:
!pip install ortools
!pip install absl-py
!pip install protobuf

In [3]:
from google.protobuf import text_format
from ortools.sat.python import cp_model
from absl import app, flags


### Constraint

Clase para métodos estáticos relacionados con restricciones.

In [6]:
class Constraint:
    """Clase para métodos estáticos relacionados con restricciones."""

    @staticmethod
    def negated_bounded_span(
            works: list[cp_model.BoolVarT], start: int, length: int
    ) -> list[cp_model.BoolVarT]:
        """Filters an isolated sub-sequence of variables assined to True.

        Extract the span of Boolean variables [start, start + length), negate them,
        and if there is variables to the left/right of this span, surround the span by
        them in non negated form.

        Args:
          works: a list of variables to extract the span from.
          start: the start to the span.
          length: the length of the span.

        Returns:
          a list of variables which conjunction will be false if the sub-list is
          assigned to True, and correctly bounded by variables assigned to False,
          or by the start or end of works.
        """
        sequence = []
        # left border (start of works, or works[start - 1])
        if start > 0:
            sequence.append(works[start - 1])
        for i in range(length):
            sequence.append(~works[start + i])
        # right border (end of works or works[start + length])
        if start + length < len(works):
            sequence.append(works[start + length])
        return sequence

    @staticmethod
    def add_soft_sequence_constraint(
            model: cp_model.CpModel,
            works: list[cp_model.BoolVarT],
            hard_min: int,
            soft_min: int,
            min_cost: int,
            soft_max: int,
            hard_max: int,
            max_cost: int,
            prefix: str,
    ) -> tuple[list[cp_model.BoolVarT], list[int]]:
        """Sequence constraint on true variables with soft and hard bounds.

        This constraint look at every maximal contiguous sequence of variables
        assigned to true. If forbids sequence of length < hard_min or > hard_max.
        Then it creates penalty terms if the length is < soft_min or > soft_max.

        Args:
          model: the sequence constraint is built on this model.
          works: a list of Boolean variables.
          hard_min: any sequence of true variables must have a length of at least
            hard_min.
          soft_min: any sequence should have a length of at least soft_min, or a
            linear penalty on the delta will be added to the objective.
          min_cost: the coefficient of the linear penalty if the length is less than
            soft_min.
          soft_max: any sequence should have a length of at most soft_max, or a linear
            penalty on the delta will be added to the objective.
          hard_max: any sequence of true variables must have a length of at most
            hard_max.
          max_cost: the coefficient of the linear penalty if the length is more than
            soft_max.
          prefix: a base name for penalty literals.

        Returns:
          a tuple (variables_list, coefficient_list) containing the different
          penalties created by the sequence constraint.
        """
        cost_literals = []
        cost_coefficients = []

        # Forbid sequences that are too short.
        for length in range(1, hard_min):
            for start in range(len(works) - length + 1):
                model.add_bool_or(Constraint.negated_bounded_span(works, start, length))

        # Penalize sequences that are below the soft limit.
        if min_cost > 0:
            for length in range(hard_min, soft_min):
                for start in range(len(works) - length + 1):
                    span = Constraint.negated_bounded_span(works, start, length)
                    name = f": under_span(start={start}, length={length})"
                    lit = model.new_bool_var(prefix + name)
                    span.append(lit)
                    model.add_bool_or(span)
                    cost_literals.append(lit)
                    # We filter exactly the sequence with a short length.
                    # The penalty is proportional to the delta with soft_min.
                    cost_coefficients.append(min_cost * (soft_min - length))

        # Penalize sequences that are above the soft limit.
        if max_cost > 0:
            for length in range(soft_max + 1, hard_max + 1):
                for start in range(len(works) - length + 1):
                    span = Constraint.negated_bounded_span(works, start, length)
                    name = f": over_span(start={start}, length={length})"
                    lit = model.new_bool_var(prefix + name)
                    span.append(lit)
                    model.add_bool_or(span)
                    cost_literals.append(lit)
                    # Cost paid is max_cost * excess length.
                    cost_coefficients.append(max_cost * (length - soft_max))

        # Just forbid any sequence of true variables with length hard_max + 1
        for start in range(len(works) - hard_max):
            model.add_bool_or([~works[i] for i in range(start, start + hard_max + 1)])
        return cost_literals, cost_coefficients

    @staticmethod
    def add_soft_sum_constraint(
            model: cp_model.CpModel,
            works: list[cp_model.BoolVarT],
            hard_min: int,
            soft_min: int,
            min_cost: int,
            soft_max: int,
            hard_max: int,
            max_cost: int,
            prefix: str,
    ) -> tuple[list[cp_model.IntVar], list[int]]:
        """sum constraint with soft and hard bounds.

        This constraint counts the variables assigned to true from works.
        If forbids sum < hard_min or > hard_max.
        Then it creates penalty terms if the sum is < soft_min or > soft_max.

        Args:
          model: the sequence constraint is built on this model.
          works: a list of Boolean variables.
          hard_min: any sequence of true variables must have a sum of at least
            hard_min.
          soft_min: any sequence should have a sum of at least soft_min, or a linear
            penalty on the delta will be added to the objective.
          min_cost: the coefficient of the linear penalty if the sum is less than
            soft_min.
          soft_max: any sequence should have a sum of at most soft_max, or a linear
            penalty on the delta will be added to the objective.
          hard_max: any sequence of true variables must have a sum of at most
            hard_max.
          max_cost: the coefficient of the linear penalty if the sum is more than
            soft_max.
          prefix: a base name for penalty variables.

        Returns:
          a tuple (variables_list, coefficient_list) containing the different
          penalties created by the sequence constraint.
        """
        cost_variables = []
        cost_coefficients = []
        sum_var = model.new_int_var(hard_min, hard_max, "")
        # This adds the hard constraints on the sum.
        model.add(sum_var == sum(works))

        # Penalize sums below the soft_min target.
        if soft_min > hard_min and min_cost > 0:
            delta = model.new_int_var(-len(works), len(works), "")
            model.add(delta == soft_min - sum_var)
            # TODO(user): Compare efficiency with only excess >= soft_min - sum_var.
            excess = model.new_int_var(0, 7, prefix + ": under_sum")
            model.add_max_equality(excess, [delta, 0])
            cost_variables.append(excess)
            cost_coefficients.append(min_cost)

        # Penalize sums above the soft_max target.
        if soft_max < hard_max and max_cost > 0:
            delta = model.new_int_var(-7, 7, "")
            model.add(delta == sum_var - soft_max)
            excess = model.new_int_var(0, 7, prefix + ": over_sum")
            model.add_max_equality(excess, [delta, 0])
            cost_variables.append(excess)
            cost_coefficients.append(max_cost)

        return cost_variables, cost_coefficients


### Model

Encapsula la creación del modelo de programación de turnos.

In [9]:
class Model:
    """Encapsula la creación del modelo de programación de turnos."""

    def __init__(self, num_employees, num_weeks):
        # Datos básicos para la planificación.
        self.num_employees = num_employees
        self.num_weeks = num_weeks
        self.num_days = num_weeks * 7
        self.day_names = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        self.shifts = ["Off", "Morning", "Afternoon", "Night"] #["O", "M", "A", "N"]
        self.num_shifts = len(self.shifts)
        self.model = cp_model.CpModel()
        self.work = {}

        self.obj_int_vars = []
        self.obj_int_coeffs = []
        self.obj_bool_vars = []
        self.obj_bool_coeffs = []

    def initialize_variables(self):
        """Inicializa las variables de trabajo (shift assignments)."""
        for e in range(self.num_employees):
            for s in range(self.num_shifts):
                for d in range(self.num_days):
                    self.work[e, s, d] = self.model.new_bool_var(f"work{e}_{s}_{d}")

    def add_constraints(self, fixed_assignments, requests, shift_constraints,
                        weekly_constraints, penalized_transitions, weekly_cover_demands, excess_cover_penalties):
        """Agrega las restricciones al modelo."""
        self.add_one_shift_per_day_constraint()
        self.add_fixed_assignments(fixed_assignments)
        self.add_employee_requests(requests)
        self.add_shift_constraints(shift_constraints)
        self.add_weekly_constraints(weekly_constraints)
        self.add_transition_constraints(penalized_transitions)
        self.add_cover_constraints(weekly_cover_demands, excess_cover_penalties)

    def add_one_shift_per_day_constraint(self):
        """Asegura que cada empleado tenga exactamente un turno por día."""
        for e in range(self.num_employees):
            for d in range(self.num_days):
                self.model.add_exactly_one(self.work[e, s, d] for s in range(self.num_shifts))

    def add_fixed_assignments(self, fixed_assignments):
        """Asigna turnos fijos según las restricciones."""
        for e, s, d in fixed_assignments:
            self.model.add(self.work[e, s, d] == 1)

    def add_employee_requests(self, requests):
        """Agrega las solicitudes de los empleados (positivas y negativas)."""
        for e, s, d, w in requests:
            self.obj_bool_vars.append(self.work[e, s, d])
            self.obj_bool_coeffs.append(w)

    def add_shift_constraints(self, shift_constraints):
        """Agrega restricciones de secuencia para turnos específicos."""
        for ct in shift_constraints:
            shift, hard_min, soft_min, min_cost, soft_max, hard_max, max_cost = ct
            for e in range(self.num_employees):
                works = [self.work[e, shift, d] for d in range(self.num_days)]
                variables, coeffs = Constraint.add_soft_sequence_constraint(
                    self.model,
                    works,
                    hard_min,
                    soft_min,
                    min_cost,
                    soft_max,
                    hard_max,
                    max_cost,
                    f"shift_constraint(employee {e}, shift {shift})",
                )
                self.obj_bool_vars.extend(variables)
                self.obj_bool_coeffs.extend(coeffs)

    def add_weekly_constraints(self, weekly_constraints):
        """Agrega restricciones semanales a los turnos."""
        for ct in weekly_constraints:
            shift, hard_min, soft_min, min_cost, soft_max, hard_max, max_cost = ct
            for e in range(self.num_employees):
                for w in range(self.num_weeks):
                    works = [self.work[e, shift, d + w * 7] for d in range(7)]
                    variables, coeffs = Constraint.add_soft_sum_constraint(
                        self.model,
                        works,
                        hard_min,
                        soft_min,
                        min_cost,
                        soft_max,
                        hard_max,
                        max_cost,
                        f"weekly_sum_constraint(employee {e}, shift {shift}, week {w})",
                    )
                    self.obj_int_vars.extend(variables)
                    self.obj_int_coeffs.extend(coeffs)

    def add_transition_constraints(self, penalized_transitions):
        """Agrega restricciones de transiciones penalizadas entre turnos."""
        for previous_shift, next_shift, cost in penalized_transitions:
            for e in range(self.num_employees):
                for d in range(self.num_days - 1):
                    transition = [
                        ~self.work[e, previous_shift, d],
                        ~self.work[e, next_shift, d + 1],
                    ]
                    if cost == 0:
                        self.model.add_bool_or(transition)
                    else:
                        trans_var = self.model.new_bool_var(
                            f"transition (employee={e}, day={d})"
                        )
                        transition.append(trans_var)
                        self.model.add_bool_or(transition)
                        self.obj_bool_vars.append(trans_var)
                        self.obj_bool_coeffs.append(cost)

    def add_cover_constraints(self, weekly_cover_demands, excess_cover_penalties):
        """Asegura que se cumplan las demandas mínimas de turnos."""
        for s in range(1, self.num_shifts):  # Ignoramos el turno "Off" (0)
            for w in range(self.num_weeks):
                for d in range(7):
                    works = [self.work[e, s, w * 7 + d] for e in range(self.num_employees)]
                    min_demand = weekly_cover_demands[d][s - 1]
                    worked = self.model.new_int_var(min_demand, self.num_employees, "")
                    self.model.add(worked == sum(works))
                    over_penalty = excess_cover_penalties[s - 1]
                    if over_penalty > 0:
                        name = f"excess_demand(shift={s}, week={w}, day={d})"
                        excess = self.model.new_int_var(0, self.num_employees - min_demand, name)
                        self.model.add(excess == worked - min_demand)
                        self.obj_int_vars.append(excess)
                        self.obj_int_coeffs.append(over_penalty)

    def set_objective(self):
        """Configura la minimización del objetivo."""
        self.model.minimize(
            sum(self.obj_bool_vars[i] * self.obj_bool_coeffs[i] for i in range(len(self.obj_bool_vars)))
            + sum(self.obj_int_vars[i] * self.obj_int_coeffs[i] for i in range(len(self.obj_int_vars)))
        )


### Solver

Se encarga de resolver el problema del modelo.

In [16]:
class Solver:
    """Se encarga de resolver el problema del modelo."""

    def __init__(self, model):
        self.model = model
        self.solver = cp_model.CpSolver()

    def solve(self, params, output_proto):
        """Resuelve el modelo especificado."""
        if output_proto:
            print(f"Writing proto to {output_proto}")
            with open(output_proto, "w") as text_file:
                text_file.write(str(self.model.model))
        if params:
            text_format.Parse(params, self.solver.parameters)
        solution_printer = cp_model.ObjectiveSolutionPrinter()
        return self.solver.solve(self.model.model, solution_printer)

    def print_solution(self, status):
        """Imprime la solución en caso de ser óptima o factible."""
        '''if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            print()
    
            # Imprime por semana
            for week in range(self.model.num_weeks):
                print(f"Week {week + 1}")
    
                # Encabezado con nombres de los días
                header = " " * 15  # Espaciado inicial para alineación correcta
                header += " ".join(f"{day:<10}" for day in self.model.day_names)
                print(header)
    
                # Imprime el horario de cada empleado para la semana actual
                for e in range(self.model.num_employees):
                    schedule = ""
                    for d in range(week * 7, (week + 1) * 7):  # Días de la semana actual
                        for s in range(self.model.num_shifts):
                            if self.solver.boolean_value(self.model.work[e, s, d]):
                                schedule += f"{self.model.shifts[s]:<11}"
                                break
                    print(f"{Mappings.ID_TO_WORKER[e]:<15}{schedule}")
    
                print()'''

        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            print()
            # Imprime por semana
            for week in range(self.model.num_weeks):
                print(f"Week {week + 1}")
                # Encabezado con nombres de los días
                header = " " * 15 + "| "  # Espaciado inicial para alineación correcta
                header += " | ".join(f"{day:<11}" for day in self.model.day_names)
                header += " |"
                print(header)
                print("-" * len(header))  # Línea horizontal que cruza todo el ancho del encabezado

                # Imprime los turnos para cada día de la semana
                previous_group = None  # Para comprobar el grupo de turnos
                for shift_index, shift_name in enumerate(self.model.shifts):
                    # Ignorar el turno "Off"
                    if shift_name.lower() == "off":
                        continue

                    # Separación entre grupos de turnos
                    current_group = "Morning" if "Morning" in shift_name else \
                        "Afternoon" if "Afternoon" in shift_name else \
                            "Night" if "Night" in shift_name else "Other"
                    if current_group != previous_group and previous_group is not None:
                        print("-" * len(header))  # Línea de separación
                    previous_group = current_group

                    # Construir filas de empleados asignados para cada día
                    rows = [[] for _ in range(7)]  # 7 días en una semana
                    max_lines = 1  # Máximo número de líneas en cualquier columna
                    for d in range(week * 7, (week + 1) * 7):  # Días de la semana actual
                        # Encuentra empleados asignados al turno actual
                        employees = [
                            Mappings.ID_TO_WORKER[e]
                            for e in range(self.model.num_employees)
                            if self.solver.boolean_value(self.model.work[e, shift_index, d])
                        ]
                        rows[d % 7] = employees
                        max_lines = max(max_lines, len(employees))

                    # Imprime línea por línea para el turno
                    for line in range(max_lines):
                        row = f"{shift_name if line == 0 else '':<15}" + "| "  # Nombre del turno solo en la primera línea
                        for day in range(7):  # Repite por los 7 días
                            if line < len(rows[day]):  # Hay un empleado asignado en esta línea
                                row += f"{rows[day][line]:<11} | "
                            else:  # Espacio vacío
                                row += " " * 11 + " | "
                        print(row)

                print("-" * len(header))  # Línea horizontal al final de cada semana
                print()  # Línea vacía entre semanas

            # Imprimir penalizaciones
            print("\nPenalties:")
            for i, var in enumerate(self.model.obj_bool_vars):
                if self.solver.boolean_value(var):
                    penalty = self.model.obj_bool_coeffs[i]
                    if penalty > 0:
                        print(f"  {var.name} violated, penalty={penalty}")
                    else:
                        print(f"  {var.name} fulfilled, gain={-penalty}")

            for i, int_var in enumerate(self.model.obj_int_vars):
                if self.solver.value(int_var) > 0:
                    penalty = self.model.obj_int_coeffs[i]
                    print(f"  {int_var.Name()} violated by {self.solver.value(int_var)}, linear penalty={penalty}")

        # Agregar el resumen de la respuesta del solver
        print(self.solver.ResponseStats())

#### Mappings (for data conversions)

In [19]:
class Mappings:
    SHIFT = {"libre": 0, "mañana": 1, "tarde": 2, "noche": 3}
    DAYS = {"lunes": 0, "martes": 1, "miércoles": 2, "jueves": 3, "viernes": 4, "sábado": 5, "domingo": 6}
    EMPLOYEES = {
        "Pepito": 0, "Juanita": 1, "Carlos": 2, "Ana": 3,
        "Maria": 4, "Miguel": 5, "Juan": 6, "Sara": 7
    }
    ID_TO_WORKER = {v: k for k, v in EMPLOYEES.items()}

#### Data conversions

In [22]:
def create_request(employee_name: str, shift: str, day_of_week: str, weight: int) -> tuple:
    if employee_name not in Mappings.EMPLOYEES:
        raise ValueError(f"Empleado '{employee_name}' no encontrado.")
    if shift.lower() not in Mappings.SHIFT:
        raise ValueError(f"Turno '{shift}' no es válido. Usa 'mañana', 'tarde' o 'noche'.")
    if day_of_week.lower() not in Mappings.DAYS:
        raise ValueError(f"Día '{day_of_week}' no reconocido. Usa: {', '.join(Mappings.DAYS.keys())}")

    return (
    Mappings.EMPLOYEES[employee_name], Mappings.SHIFT[shift.lower()], Mappings.DAYS[day_of_week.lower()], weight)


def create_fixed_assignment(employee_name: str, shift: str, day_of_week: str) -> tuple:
    if employee_name not in Mappings.EMPLOYEES:
        raise ValueError(f"Empleado '{employee_name}' no encontrado.")
    if shift.lower() not in Mappings.SHIFT:
        raise ValueError(f"Turno '{shift}' no es válido. Usa 'mañana', 'tarde' o 'noche'.")
    if day_of_week.lower() not in Mappings.DAYS:
        raise ValueError(f"Día '{day_of_week}' no reconocido. Usa: {', '.join(Mappings.DAYS.keys())}")

    return (Mappings.EMPLOYEES[employee_name], Mappings.SHIFT[shift.lower()], Mappings.DAYS[day_of_week.lower()])


def create_shift_constraint(shift: str, hard_min: int, soft_min: int, min_penalty: int, soft_max: int, hard_max: int,
                            max_penalty: int) -> tuple:
    if shift.lower() not in Mappings.SHIFT:
        raise ValueError(f"Turno '{shift}' no es válido. Usa 'libre', 'mañana', 'tarde' o 'noche'.")

    return (Mappings.SHIFT[shift.lower()], hard_min, soft_min, min_penalty, soft_max, hard_max, max_penalty)


def create_weekly_sum_constraint(shift: str, hard_min: int, soft_min: int, min_penalty: int, soft_max: int,
                                 hard_max: int, max_penalty: int) -> tuple:
    if shift.lower() not in Mappings.SHIFT:
        raise ValueError(f"Turno '{shift}' no es válido. Usa 'libre', 'mañana', 'tarde' o 'noche'.")

    return (Mappings.SHIFT[shift.lower()], hard_min, soft_min, min_penalty, soft_max, hard_max, max_penalty)


def create_penalized_transition(previous_shift: str, next_shift: str, penalty: int) -> tuple:
    if previous_shift.lower() not in Mappings.SHIFT:
        raise ValueError(f"Turno previo '{previous_shift}' no válido.")
    if next_shift.lower() not in Mappings.SHIFT:
        raise ValueError(f"Turno siguiente '{next_shift}' no válido.")

    return (Mappings.SHIFT[previous_shift.lower()], Mappings.SHIFT[next_shift.lower()], penalty)


def create_daily_demand(morning: int, afternoon: int, night: int) -> tuple:
    # No hay mappings necesarios aquí, pero el formato del retorno es consistente con el diseño.
    return (morning, afternoon, night)

## Main

In [25]:
# Definiciones de parámetros (sin uso de flags de terminal)
output_proto = ""  # Archivo para escribir el proto (opcional)
params = "max_time_in_seconds:3.0"  # Parámetros del solver SAT

# Función principal adaptada
def main():
    # Datos originales del problema
    num_employees = 8
    num_weeks = 3

    fixed_assignments = [
        create_fixed_assignment("Pepito", "libre", "lunes"),
        create_fixed_assignment("Juanita", "libre", "lunes"),
        create_fixed_assignment("Carlos", "mañana", "lunes"),
        create_fixed_assignment("Ana", "mañana", "lunes"),
        create_fixed_assignment("Maria", "tarde", "lunes"),
        create_fixed_assignment("Miguel", "tarde", "lunes"),
        create_fixed_assignment("Juan", "tarde", "jueves"),
        create_fixed_assignment("Sara", "noche", "lunes"),
        create_fixed_assignment("Pepito", "mañana", "martes"),
        create_fixed_assignment("Juanita", "mañana", "martes"),
        create_fixed_assignment("Carlos", "tarde", "martes"),
        create_fixed_assignment("Ana", "tarde", "martes"),
        create_fixed_assignment("Maria", "tarde", "martes"),
        create_fixed_assignment("Miguel", "libre", "martes"),
        create_fixed_assignment("Juan", "libre", "martes"),
        create_fixed_assignment("Sara", "noche", "martes"),
    ]

    requests = [
        create_request("Ana", "libre", "sábado", -2),
        create_request("Miguel", "noche", "jueves", -2),
        create_request("Carlos", "noche", "viernes", 4),
    ]

    shift_constraints = [
        create_shift_constraint("libre", 1, 1, 0, 2, 2, 0),
        create_shift_constraint("noche", 1, 2, 20, 3, 4, 5),
    ]

    weekly_constraints = [
        create_weekly_sum_constraint("libre", 1, 2, 7, 2, 3, 4),
        create_weekly_sum_constraint("noche", 0, 1, 3, 4, 4, 0),
    ]

    penalized_transitions = [
        create_penalized_transition("tarde", "noche", 4),
        create_penalized_transition("noche", "mañana", 0),
    ]

    weekly_cover_demands = [
        create_daily_demand(2, 3, 1),  # Lunes
        create_daily_demand(2, 3, 1),  # Martes
        create_daily_demand(2, 2, 2),  # Miércoles
        create_daily_demand(2, 3, 1),  # Jueves
        create_daily_demand(2, 2, 2),  # Viernes
        create_daily_demand(1, 2, 3),  # Sábado
        create_daily_demand(1, 3, 1),  # Domingo
    ]

    excess_cover_penalties = (2, 2, 5)

    # Inicializar modelo
    model = Model(num_employees, num_weeks)
    model.initialize_variables()
    model.add_constraints(fixed_assignments, requests, shift_constraints, weekly_constraints, penalized_transitions,
                          weekly_cover_demands, excess_cover_penalties)

    # Configurar objetivo
    model.set_objective()

    # Resolver modelo
    solver = Solver(model)
    status = solver.solve(params, output_proto)
    solver.print_solution(status)

# Ejecutar la función principal
if __name__ == "__main__":
    main()


Solution 0, time = 0.24 s, objective = 311
Solution 1, time = 0.27 s, objective = 267
Solution 2, time = 0.28 s, objective = 140
Solution 3, time = 0.35 s, objective = 131
Solution 4, time = 0.36 s, objective = 127
Solution 5, time = 0.39 s, objective = 124
Solution 6, time = 0.40 s, objective = 120
Solution 7, time = 0.43 s, objective = 116
Solution 8, time = 0.48 s, objective = 114
Solution 9, time = 0.50 s, objective = 105
Solution 10, time = 0.55 s, objective = 102
Solution 11, time = 0.60 s, objective = 93
Solution 12, time = 0.61 s, objective = 90
Solution 13, time = 0.63 s, objective = 81
Solution 14, time = 0.67 s, objective = 78
Solution 15, time = 0.74 s, objective = 72
Solution 16, time = 0.76 s, objective = 68
Solution 17, time = 0.82 s, objective = 66
Solution 18, time = 0.89 s, objective = 63
Solution 19, time = 1.08 s, objective = 60
Solution 20, time = 1.20 s, objective = 58
Solution 21, time = 1.22 s, objective = 53
Solution 22, time = 1.27 s, objective = 51
Solution 2